### Download and Prepare the Dog Cats Dataset

In [1]:
#!/bin/bash
!curl -L -o cats-and-dogs-mini-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/aleemaparakatta/cats-and-dogs-mini-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0 21.8M    0 38522    0     0  28178      0  0:13:33  0:00:01  0:13:32 28178
  1 21.8M    1  414k    0     0   174k      0  0:02:08  0:00:02  0:02:06  373k
 16 21.8M   16 3584k    0     0  1053k      0  0:00:21  0:00:03  0:00:18 1742k
 32 21.8M   32 7341k    0     0  1650k      0  0:00:13  0:00:04  0:00:09 2371k
 42 21.8M   42 9629k    0     0  1788k      0  0:00:12  0:00:05  0:00:07 2387k
 50 21.8M   50 10.9M    0     0  1767k      0  0:00:12  0:00:06  0:00:06 2242k
 64 21.8M   64 14.0M    0     0  1934k      0  0:00:11  0:00:07  0:00:04 2760k
 76 21.8M   76 16.6M    0     0  1994k      0  0:0

### Creating Dataset Structure for Training and Inferencing

In [3]:
import zipfile
from pathlib import Path

zip_path = "cats-and-dogs-mini-dataset.zip"
extract_path = Path("dataset")

extract_path.mkdir(exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete ✅")


Extraction complete ✅


In [4]:
import os

# Create the 'data' directory if it doesn't exist
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Create the 'train' and 'test' subdirectories within 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [5]:
import os

# Define the source and destination paths
source_cat = 'dataset/cats_set'
dest_cat = 'dataset/cat'
source_dog = 'dataset/dogs_set'
dest_dog = 'dataset/dog'

# Rename the directories if they exist
if os.path.exists(source_cat):
  os.rename(source_cat, dest_cat)

if os.path.exists(source_dog):
  os.rename(source_dog, dest_dog)


### Creating the TRAIN TEST SPLIT within the DATA directory folders

In [6]:
import os
import random
import shutil

def train_test_split_folder(source_folder, train_folder, test_folder, split_ratio=0.8):
    """
    Splits a folder of images into training and testing sets.

    Args:
        source_folder: Path to the source folder containing subfolders for each class.
        train_folder: Path to the folder where the training set will be saved.
        test_folder: Path to the folder where the testing set will be saved.
        split_ratio: The ratio of images to include in the training set (default is 0.8).
    """

    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    for class_name in os.listdir(source_folder):
        class_source_path = os.path.join(source_folder, class_name)

        if os.path.isdir(class_source_path):  # Check if it is a directory
            train_class_path = os.path.join(train_folder, class_name)
            test_class_path = os.path.join(test_folder, class_name)

            if not os.path.exists(train_class_path):
                os.makedirs(train_class_path)
            if not os.path.exists(test_class_path):
                os.makedirs(test_class_path)

            images = [f for f in os.listdir(class_source_path) if os.path.isfile(os.path.join(class_source_path, f))]
            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            for image in train_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(train_class_path, image)
                shutil.copy(source_path, destination_path)

            for image in test_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(test_class_path, image)
                shutil.copy(source_path, destination_path)


# Example usage (assuming you have your data organized in a 'data/train' folder):
train_test_split_folder("dataset", "data/train", "data/test")

### Counting total Images in Folders

In [7]:
import os

def count_images_per_folder(root_folder):
  """
  Counts the number of images in each subfolder of a given root folder.

  Args:
    root_folder: The path to the root folder.

  Returns:
    A dictionary where keys are folder paths and values are the number of images in each folder.
  """

  image_counts = {}
  for dirpath, dirnames, filenames in os.walk(root_folder):
    image_count = 0
    for filename in filenames:
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_count += 1
    if image_count > 0 :
      image_counts[dirpath] = image_count
  return image_counts

# Example usage
image_counts = count_images_per_folder("data")
for folder, count in image_counts.items():
    print(f"Folder: {folder}, Number of images: {count}")


Folder: data\test\cat, Number of images: 100
Folder: data\test\dog, Number of images: 100
Folder: data\train\cat, Number of images: 400
Folder: data\train\dog, Number of images: 400


### Necesary Imports

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

### Set paths to train and test folders

In [9]:
# Set paths to train and test folders
train_dir = "data/train"
test_dir = "data/test"
image_size = (128, 128)
batch_size = 32

### Data Preparation

In [10]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 640 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


### Build the CNN Model

In [17]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

### Model Compilation & Training

In [24]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the Model
checkpoint_path = 'dog_cat_cnn_model.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True)

model.fit(
    train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=[checkpoint]
)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.7359 - loss: 0.5302 - val_accuracy: 0.7500 - val_loss: 0.5790
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 222ms/step - accuracy: 0.7641 - loss: 0.5066 - val_accuracy: 0.6500 - val_loss: 0.5844
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 295ms/step - accuracy: 0.7984 - loss: 0.4800 - val_accuracy: 0.6750 - val_loss: 0.5860
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7937 - loss: 0.4593 - val_accuracy: 0.6750 - val_loss: 0.5753
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 202ms/step - accuracy: 0.7953 - loss: 0.4500 - val_accuracy: 0.7000 - val_loss: 0.5838
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 259ms/step - accuracy: 0.8375 - loss: 0.4199 - val_accuracy: 0.7500 - val_loss: 0.5817
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 220ms/step - accuracy: 0.8141 - loss: 0.4175 - val_accuracy: 0.7000 - val_loss: 0.5800
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - accuracy: 0.8391 - loss: 0.4007 - val_accuracy: 0.

### Saving the Trained Model

In [25]:
# Save final model
model.save('model1.keras')

### Loading the Trained Model

In [26]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('model1.keras')

### Model Evaluation

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
val_dir = test_dir  # same directory used for training
image_size = (128, 128)
batch_size = 32

# Recreate the validation generator
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Evaluate model accuracy
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 50 images belonging to 2 classes.


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7800 - loss: 0.6615 
Validation Accuracy: 78.00%


### Model Inferencing

In [28]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        print("Predicted: Dog")
    else:
        print("Predicted: Cat")

In [37]:
predict_image("data/train/cat/cat.4063.jpg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted: Cat


In [43]:
predict_image("data/test/dog/dog.4096.jpg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted: Dog
